In [1]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn

import torch
import torch.nn as nn

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        

        self.encoder1 = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = 5),  
            nn.ReLU(),
            nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5),
        )

        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(in_channels = 32, out_channels= 16, kernel_size = 5),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels = 16, out_channels= 1, kernel_size = 5),
            nn.Sigmoid(),
        )

    def forward(self, x, return_features=False):
        encoded1 = self.encoder1(x)
        if return_features:
            return encoded1  
        decoded1 = self.decoder1(encoded1)
        return decoded1 

In [2]:
# Resim yükleme ve dönüştürme işlemleri
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),  # Resimleri 28x28 boyutuna küçültüyoruz
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize etme işlemi (Sonradan ekledim.)
])

# Train ve Test datasetlerini yükleme
train_data = datasets.ImageFolder(root='../train', transform=transform)
test_data = datasets.ImageFolder(root='../test', transform=transform)

# DataLoader ile veri kümelerini yükleme
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=True)

# Modeli başlatma
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Autoencoder().to(device)

In [3]:
import torch.optim as optim

# Eğitim ayarları
num_epochs = 100
#learning_rate = 0.001

# Kayıp fonksiyonu ve optimizer
criterion = nn.MSELoss()  # Mean Squared Error kaybı
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Modeli eğitme fonksiyonu
def train_autoencoder(model, train_loader, criterion, optimizer, num_epochs, patience=5, delta=0.001):
    model.train()  # Modeli eğitim moduna alıyoruz
    
    best_loss = float('inf')  # En iyi kayıp değeri
    patience_counter = 0  # Patience sayacı

    for epoch in range(num_epochs):
        running_loss = 0.0
        
        for data, _ in train_loader:  # Giriş verisi ve etiketler (etiketler kullanılmaz çünkü reconstruct ediliyor)
            data = data.to(device)
            
            # İleri yayılım (forward pass)
            output = model(data)
            
            # Kayıp hesaplama
            loss = criterion(output, data)  # Girdi veri ile çıktı veri arasındaki fark
            
            # Geri yayılım (backpropagation) ve optimizasyon
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Her epoch sonunda ortalama kaybı yazdırma
        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Early stopping kontrolü
        if epoch_loss < best_loss - delta:
            best_loss = epoch_loss  # En iyi loss güncelleniyor
            patience_counter = 0  # Patience sayacı sıfırlanıyor
        else:
            patience_counter += 1  # Eğer gelişme yoksa patience artırılır
        
        print(f"Patience counter of epoch {epoch+1} is {patience_counter}")
        
        # Eğer patience counter belirlenen değeri aşarsa durdur
        if patience_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break


# Modeli eğitme
train_autoencoder(model, train_loader, criterion, optimizer, num_epochs)

Epoch [1/100], Loss: 0.0542
Patience counter of epoch 1 is 0
Epoch [2/100], Loss: 0.0316
Patience counter of epoch 2 is 0
Epoch [3/100], Loss: 0.0282
Patience counter of epoch 3 is 0
Epoch [4/100], Loss: 0.0269
Patience counter of epoch 4 is 0
Epoch [5/100], Loss: 0.0262
Patience counter of epoch 5 is 1
Epoch [6/100], Loss: 0.0257
Patience counter of epoch 6 is 0
Epoch [7/100], Loss: 0.0253
Patience counter of epoch 7 is 1
Epoch [8/100], Loss: 0.0251
Patience counter of epoch 8 is 2
Epoch [9/100], Loss: 0.0249
Patience counter of epoch 9 is 3
Epoch [10/100], Loss: 0.0247
Patience counter of epoch 10 is 4
Epoch [11/100], Loss: 0.0246
Patience counter of epoch 11 is 0
Epoch [12/100], Loss: 0.0245
Patience counter of epoch 12 is 1
Epoch [13/100], Loss: 0.0244
Patience counter of epoch 13 is 2
Epoch [14/100], Loss: 0.0244
Patience counter of epoch 14 is 3
Epoch [15/100], Loss: 0.0243
Patience counter of epoch 15 is 4
Epoch [16/100], Loss: 0.0242
Patience counter of epoch 16 is 5
Early stop

In [4]:
def extract_features(loader, model):
    model.eval()  # Modeli eval moduna alıyoruz
    features = []
    labels = []
    
    with torch.no_grad():  
        for data, label in loader:
            data = data.to(device)
            output = model(data, return_features=True)  
            output = output.view(output.size(0), -1)  
            features.append(output.cpu().numpy())
            labels.append(label.cpu().numpy())
    
    features = np.vstack(features)
    labels = np.hstack(labels)
    
    return features, labels

In [5]:
# Train ve Test veri setlerinden özellikler çıkarıyoruz
train_features, train_labels = extract_features(train_loader, model)
test_features, test_labels = extract_features(test_loader, model)

# Özellik ve etiket dosyalarını .npy olarak kaydetme
np.save('train_features_autoencoder.npy', train_features)
np.save('train_labels_autoencoder.npy', train_labels)
np.save('test_features_autoencoder.npy', test_features)
np.save('test_labels_autoencoder.npy', test_labels)

print("Train ve Test veri setleri başarıyla kaydedildi.")

Train ve Test veri setleri başarıyla kaydedildi.


In [6]:
# Modeli kaydetme
torch.save(model.state_dict(), 'model_ae.pth')